In [ ]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import optuna
import importlib

sys.path.append(os.path.abspath(".."))

import src.models.lgbm.lgbm_cv_trainer as cv
import src.models.lgbm.lgbm_optuna_optimizer as op
import src.utils.telegram as te

In [ ]:
# Load data
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

tr_df1 = pd.read_parquet("../artifacts/features/base/tr_df1.parquet")

In [ ]:
# Tuning
importlib.reload(cv)
importlib.reload(op)
objective = op.create_objective(
    tr_df1,
    early_stopping_rounds=500,
    n_jobs=25
)

study = op.run_optuna_search(
    objective,
    n_trials=50,
    n_jobs=1,
    study_name="lgbm_v1",
    storage=url,
    sampler=optuna.samplers.TPESampler(
        n_startup_trials=20, seed=42)
)
te.send_telegram_message("LGBM Training Complete!")